# Variant effect prediction - simple

NOTE: This notebook is a companion notebook to `variant_effect_prediction.ipynb` and shows a simpler way to run variant effect predictions in python. If you want to know more details about variant effect prediction, the details of how to customise things and how to run it on the command line and in batch please refer to the `variant_effect_prediction.ipynb`.

Variant effect prediction offers a simple way to predict effects of SNVs using any model that uses DNA sequence as an input. Many different scoring methods can be chosen, but the principle relies on in-silico mutagenesis. The default input is a VCF and the default output again is a VCF annotated with predictions of variant effects. 

First we set up the model and make sure the requirements are installed in the current environment.

In [1]:
import kipoi
model_name = "DeepSEA/variantEffects"
kipoi.pipeline.install_model_requirements(model_name)

Conda dependencies to be installed:
['h5py', 'pytorch::pytorch-cpu>=0.2.0']
Solving environment: ...working... done

# All requested packages already installed.

pip dependencies to be installed:
[]
Conda dependencies to be installed:
['python=3.5', 'numpy', 'pandas', 'cython']
Solving environment: ...working... done

# All requested packages already installed.

pip dependencies to be installed:
['genomelake', 'pybedtools']


Then we need to know where the query VCF is located and where we want to store the results.

In [2]:
# The input vcf path
vcf_path = "example_data/clinvar_donor_acceptor_chr22.vcf"
# The output vcf path, based on the input file name    
out_vcf_fpath = vcf_path[:-4] + "%s.vcf"%model_name.replace("/", "_")

Finally the dataloader arguments are set that are required to run the dataloader. Here we omit the `intervals_file` argument of the dataloader, because that has been tagged as bed file input in the `dataloader.yaml` file, which means that `score_variants` will automatically populate that argument with a temporary bed file that is generated from the VCF in order to query every variant contained in the input VCF. ("Variant-centered approach")

In [3]:
# The datalaoder keyword arguments
dataloader_arguments = {"fasta_file": "example_data/hg19_chr22.fa"}

In [4]:
!ls example_data/

chr22.101bp.2000_intervals.JUND.HepG2.tsv
clinvar_donor_acceptor_annotated_chr22.csv
clinvar_donor_acceptor_annotated_w_rbp_chr22.csv
clinvar_donor_acceptor_chr22DeepSEA_variantEffects.vcf
clinvar_donor_acceptor_chr22HAL.vcf
clinvar_donor_acceptor_chr22labranchor.vcf
clinvar_donor_acceptor_chr22MaxEntScan_3prime.vcf
clinvar_donor_acceptor_chr22rbp_eclip_AARS.vcf
clinvar_donor_acceptor_chr22.vcf
clinvar_donor_acceptor_chr22.vcf.gz
clinvar_donor_acceptor_chr22.vcf.gz.tbi
hg19_chr22.fa
hg19_chr22.fa.fai
Homo_sapiens.GRCh37.75_chr22.gtf
Homo_sapiens.GRCh37.75.filtered_chr22.gtf


In [5]:
import kipoi.postprocessing.variant_effects.snv_predict as sp
sp.score_variants(model = model_name,
                  dl_args = dataloader_arguments,
                  input_vcf = vcf_path,
                  output_vcf = out_vcf_fpath)

100%|██████████| 14/14 [00:02<00:00,  4.70it/s]


Now we can have a look at the generated output:

In [6]:
# Let's print out the first 40 lines of the annotated VCF (up to 80 characters per line maximum)
with open("example_data/clinvar_donor_acceptor_chr22DeepSEA_variantEffects.vcf") as ifh:
    for i,l in enumerate(ifh):
        long_line = ""
        if len(l)>80:
            long_line = "..."
        print(l[:80].rstrip() +long_line)
        if i >=40:
            break

##fileformat=VCFv4.0
##INFO=<ID=KV:kipoi:DeepSEA/variantEffects:LOGIT_ALT,Number=.,Type=String,Descri...
##INFO=<ID=KV:kipoi:DeepSEA/variantEffects:LOGIT_REF,Number=.,Type=String,Descri...
##INFO=<ID=KV:kipoi:DeepSEA/variantEffects:DIFF,Number=.,Type=String,Description...
##INFO=<ID=KV:kipoi:DeepSEA/variantEffects:LOGIT,Number=.,Type=String,Descriptio...
##INFO=<ID=KV:kipoi:DeepSEA/variantEffects:REF,Number=.,Type=String,Description=...
##INFO=<ID=KV:kipoi:DeepSEA/variantEffects:ALT,Number=.,Type=String,Description=...
##INFO=<ID=KV:kipoi:DeepSEA/variantEffects:rID,Number=.,Type=String,Description=...
##FILTER=<ID=PASS,Description="All filters passed">
##contig=<ID=chr1,length=249250621>
##contig=<ID=chr2,length=243199373>
##contig=<ID=chr3,length=198022430>
##contig=<ID=chr4,length=191154276>
##contig=<ID=chr5,length=180915260>
##contig=<ID=chr6,length=171115067>
##contig=<ID=chr7,length=159138663>
##contig=<ID=chr8,length=146364022>
##contig=<ID=chr9,length=141213431>
##contig=<ID=ch

Here we have shown a simpler function `score_variants` that covers most use-cases for variant effect prediction. For a more fine-grain control please use `predict_snvs` and take a look at the `variant_effect_prediction.ipynb` notebook. 

An important thing to remember when using `score_variants` or the command-line interface is that for all dataloaders that support bed-file inputs, the bed file generation will be used and only model that do not have the `postprocessing > variant_effects > bed_input` field set in `dataloader.yaml` will be executed in overlap-based mode. For details on how variant region overlap works please take a look at the variant effect prediction documentation. 